<a href="https://colab.research.google.com/github/valerio-unifei/UNIFEI-IA-Aulas/blob/main/UNIFEI_IA_FOMMIA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# First Order Motion Model for Image Animation

## Preparando programa

### Instalando o código do GitHub

GitHub:

https://github.com/AliaksandrSiarohin/first-order-model

Artigo:

https://proceedings.neurips.cc/paper/2019/file/31c0b36aef265d9221af80872ceb62f9-Paper.pdf

In [1]:
!git clone https://github.com/AliaksandrSiarohin/first-order-model

Cloning into 'first-order-model'...
remote: Enumerating objects: 262, done.
remote: Counting objects: 100% (3/3), done.
remote: Compressing objects: 100% (3/3), done.
remote: Total 262 (delta 0), reused 0 (delta 0), pack-reused 259
Receiving objects: 100% (262/262), 72.13 MiB | 27.51 MiB/s, done.
Resolving deltas: 100% (131/131), done.


Após baixar código, mover para a pasta de trabalho para a raiz do código:

In [2]:
%cd /content/first-order-model/
!pwd

/content/first-order-model
/content/first-order-model


### Baixando a rede GAN pré-treinada

In [3]:
# biblioteca para download de arquivos abertos no Google Drive
!pip install gdown

In [4]:
import gdown
url = 'https://drive.google.com/uc?id=19kmpcv_kBBaCpS-F5REd_MD4p7mpOfym'
output = 'vox-cpk.pth.tar'
gdown.download(url, output, quiet=False)

Downloading...
From: https://drive.google.com/uc?id=19kmpcv_kBBaCpS-F5REd_MD4p7mpOfym
To: /content/first-order-model/vox-cpk.pth.tar
729MB [00:08, 84.1MB/s]


'vox-cpk.pth.tar'

In [5]:
from demo import load_checkpoints
generator, kp_detector = load_checkpoints(config_path='config/vox-256.yaml', 
                            checkpoint_path='vox-cpk.pth.tar')

### Rotina para exibição dos resultados obtidos

In [6]:
import imageio
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from skimage.transform import resize
from IPython.display import HTML
from demo import make_animation
from skimage import img_as_ubyte
import warnings
warnings.filterwarnings("ignore")

def display(source, driving, generated=None):
    fig = plt.figure(figsize=(8 + 4 * (generated is not None), 6))

    ims = []
    for i in range(len(driving)):
        cols = [source]
        cols.append(driving[i])
        if generated is not None:
            cols.append(generated[i])
        im = plt.imshow(np.concatenate(cols, axis=1), animated=True)
        plt.axis('off')
        ims.append([im])

    ani = animation.ArtistAnimation(fig, ims, interval=50, repeat_delay=1000)
    plt.close()
    return ani

## Exemplo

Baixando Imagem e Vídeo 

In [7]:
!wget "https://github.com/valerio-unifei/UNIFEI-IA-Aulas/raw/main/FOMMIA_Ex01.zip"
!unzip FOMMIA_Ex01.zip

--2021-04-29 21:30:18--  https://github.com/valerio-unifei/UNIFEI-IA-Aulas/raw/main/FOMMIA_Ex01.zip
Resolving github.com (github.com)... 192.30.255.112
Connecting to github.com (github.com)|192.30.255.112|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/valerio-unifei/UNIFEI-IA-Aulas/main/FOMMIA_Ex01.zip [following]
--2021-04-29 21:30:18--  https://raw.githubusercontent.com/valerio-unifei/UNIFEI-IA-Aulas/main/FOMMIA_Ex01.zip
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1565360 (1.5M) [application/zip]
Saving to: ‘FOMMIA_Ex01.zip’

FOMMIA_Ex01.zip     100%[===================>]   1.49M  --.-KB/s    in 0.07s   

2021-04-29 21:30:19 (20.0 MB/s) - ‘FOMMIA_Ex01.zip’ saved [1565360/1565360]

Archiv

Preparando os datasets para entrada na rede

In [8]:
source_image = imageio.imread('imagem.png')
reader = imageio.get_reader('video.mp4')

#Resize image and video to 256x256
source_image = resize(source_image, (256, 256))[..., :3]

fps = reader.get_meta_data()['fps']
driving_video = []
try:
    for im in reader:
        driving_video.append(im)
except RuntimeError:
    pass
reader.close()
driving_video = [resize(frame, (256, 256))[..., :3] for frame in driving_video]

HTML(display(source_image, driving_video).to_html5_video())

Realizando a animação da imagem

In [10]:
predictions = make_animation(
    source_image, 
    driving_video, 
    generator, 
    kp_detector, 
    relative=True,
    )

imageio.mimsave('exemplo01.mp4', 
                [img_as_ubyte(frame) for frame in predictions], fps=fps)

HTML(display(source_image, driving_video, predictions).to_html5_video())

100%|██████████| 240/240 [00:09<00:00, 25.65it/s]
